In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
	.master("local[*]") \
	.appName('test') \
	.getOrCreate()
	
spark.sparkContext.setLogLevel("WARN")

your 131072x1 screen size is bogus. expect trouble
24/03/02 10:57:02 WARN Utils: Your hostname, Cinders resolves to a loopback address: 127.0.1.1; using 172.17.156.62 instead (on interface eth0)
24/03/02 10:57:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/02 10:57:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green = spark.read.parquet('../../data/pq/green/*/*')

```
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
```

In [4]:
df_green.take(5)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 14, 13, 28, 9), lpep_dropoff_datetime=datetime.datetime(2020, 1, 14, 13, 35, 19), store_and_fwd_flag='N', RatecodeID=1, PULocationID=74, DOLocationID=75, passenger_count=1, trip_distance=1.35, fare_amount=7.0, extra=0.0, mta_tax=0.5, tip_amount=1.56, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=9.36, payment_type=1, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 28, 12, 11, 44), lpep_dropoff_datetime=datetime.datetime(2020, 1, 28, 12, 25, 21), store_and_fwd_flag='N', RatecodeID=1, PULocationID=129, DOLocationID=179, passenger_count=1, trip_distance=3.73, fare_amount=13.5, extra=0.0, mta_tax=0.5, tip_amount=1.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=15.3, payment_type=1, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2020, 1, 4, 11, 38), lpep_dropoff_

In [5]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [6]:
rdd.take(5)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 14, 13, 28, 9), PULocationID=74, total_amount=9.36),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 28, 12, 11, 44), PULocationID=129, total_amount=15.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 4, 11, 38), PULocationID=25, total_amount=28.88),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 30, 10, 3, 36), PULocationID=182, total_amount=12.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 30, 20, 56, 17), PULocationID=210, total_amount=7.0)]

In [7]:
rdd.filter(lambda row:True).take(1)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 14, 13, 28, 9), PULocationID=74, total_amount=9.36)]

In [8]:
from datetime import datetime

In [9]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [10]:
rows = rdd.take(10)
row = rows[0]

In [11]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 14, 13, 28, 9), PULocationID=74, total_amount=9.36)

In [12]:
def prepare_for_grouping(row): 
    # truncate to hour
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    # composite key
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)
    
    # tuple of composite key & composite value
    return (key, value)

invoke this calculate_revenue() to reduce

In [13]:
def calculate_revenue(left_value, right_value):
    # unpack the composite key
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    # unpack the composite value
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [14]:
from collections import namedtuple

column names are lost so we define it like this

In [15]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

unnested / unwrap with unwrap()

In [16]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [17]:
from pyspark.sql import types

In [18]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

the final built low-level RDD code from our original SQL

```python
df_result = rdd \
    # where
    .filter(filter_outliers)  \
    # select part, prep for groupBy 
    .map(prepare_for_grouping)  \
    # groupby and the summing + count
    .reduceByKey(calculate_revenue)  \
    # unwrap from spark format
    .map(unwrap)  \
    # to dataframe format
    .toDF(result_schema) 
```

In [23]:
df_result = rdd \
  .filter(filter_outliers) \
  .map(prepare_for_grouping) \
  .reduceByKey(calculate_revenue) \
  .map(unwrap) \
  .toDF(result_schema) 

In [24]:
df_result.show(5)

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2020-01-29 04:00:00| 116|              5.81|    1|
|2020-01-17 22:00:00|  74|            682.38|   48|
|2020-01-15 22:00:00|  82| 338.5600000000001|   27|
|2020-01-21 10:00:00|  65|417.14000000000004|   22|
|2020-01-15 01:00:00|  80|             28.79|    3|
+-------------------+----+------------------+-----+
only showing top 5 rows



In [26]:
df_result.write.parquet('../../data/tmp/green-revenue', mode="overwrite")

## MapPartition

```mermaid
graph LR;
    A[partition]-->|model|B(MapPartition)
    B-->|apply model per partition| B
    B-->|ML predictions|C[partition]

```

Say we want to predict `duration of a trip` with ML. We use the dependent variables in `columns`.

In [27]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [33]:
duration_rdd.take(2)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 14, 13, 28, 9), PULocationID=74, DOLocationID=75, trip_distance=1.35),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 28, 12, 11, 44), PULocationID=129, DOLocationID=179, trip_distance=3.73)]

How mapPartition works

- just like in Pandas.map(), it iterates over the rows
- the return value needs to be an iterable ie a list
- the len(return[1]) means there's that many partitions
- actual output is `[[1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1]]`
- and then it flattens this list (removing the nestings) to become `[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]`
- the `partition` object (being passed as args) is actually not a python list but an iterator `itertools.chain`

In [34]:
def apply_model_in_batch(partition):
    return [1]

In [36]:
rdd.mapPartitions(apply_model_in_batch).collect()

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [37]:
import pandas as pd

In [38]:
rows = duration_rdd.take(10)

In [39]:
df = pd.DataFrame(rows, columns=columns)

In [40]:
columns

['VendorID',
 'lpep_pickup_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_distance']

In [41]:
df.head(2)

,VendorID,lpep_pickup_datetime,PULocationID,DOLocationID,trip_distance
0,2.0,2020-01-14 13:28:09,74,75,1.35
1,2.0,2020-01-28 12:11:44,129,179,3.73


In [42]:
# no model, so it's a pretend step with an arbitrary function
# model = ...

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

pd.DataFrame(rows, columns=columns) is the whole DF, might result in memory issues. but I'd changed the default to 10g in conf file, taken from `spark-defaults.conf.template`

path to config file: `\spark\spark-3.5.1-bin-hadoop3\conf`

uncomment the `driver` line and add the `executor` line

```
spark.driver.memory              10g
spark.executor.memory            10g
```

Example of `yield row` using infinite_seq() below

In [48]:
def infinite_seq():
    i =0
    while True:
        yield i
        i = i +1

In [50]:
seq = infinite_seq()

In [51]:
for i in seq:
    print(i)

    if i > 10:
        break

0
1
2
3
4
5
6
7
8
9
10
11


In [43]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [52]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|              6.75|
|             18.65|
|55.949999999999996|
|6.6000000000000005|
|               4.0|
|               6.2|
|               2.6|
|               2.9|
|             33.95|
|48.449999999999996|
|              6.25|
|              6.15|
|19.950000000000003|
|2.8499999999999996|
|               7.5|
|              0.95|
|              9.05|
|              7.45|
|              5.25|
|             17.35|
+------------------+
only showing top 20 rows



Actually this would need to be a real-time service and for one customer at a time. 

Customer, the `taxi rider` input the postal code and expects the predicted duration of the trip back on their mobile

```mermaid
graph LR;
    A[taxi rider]-->|input destination postal code|B(Taxi Service)
    B-->|ML on mapPartition| B
    B-->|predict duration, eg: 20mins|A

```